In [18]:
import requests
from click.testing import CliRunner
from pprint import pprint, pformat
import pandas

from requests.auth import HTTPBasicAuth

from kf_model_fhir.config import FHIR_VERSION, SERVER_CONFIG, PROJECT_DIR
from kf_model_fhir import cli
from kf_model_fhir.client import FhirApiClient
from kf_model_fhir.loader import load_resources
from kf_model_fhir.utils import read_json
from kf_model_fhir.aidbox import *

SERVER_CONFIG.pop('simplifier', None)
runner = CliRunner()

def emoji(resp):
    total = resp.get('total', None)
    if total is None:
        return '❌'
    if total > 0:
        return '✅'
    else:
        return '⚠️ or ✅'

def execute_queries(queries, display_content=False):
    for query in queries:
        dlen = len(query['desc'])
        print('*' * dlen)
        print(query['desc'])
        print('*' * dlen)
        for server_name, settings in SERVER_CONFIG.items():
            print(server_name.upper())
            print('-' * len(server_name))
            url = f"{settings['base_url']}/{query['endpoint']}"
            uname = settings.get('username')
            pw = settings.get('password')
            params = query['params']
            params.update({'_total': 'accurate'})
            resp, status_code = get(url, username=uname, password=pw, params=params)

            if display_content:
                pprint(resp)
                
            print(f"{emoji(resp)} Found {resp.get('total')} {query['endpoint'].lstrip('/')} "
                  f"matching these params {pformat(params)}")
            if resp.get('total') == counts[query['endpoint']]:
                print(
                    f'⚠️ ？ The query count = actual total {query["endpoint"]}. '
                    f'This could be correct or the server returned all {query["endpoint"]} '
                    'because it did not understand one or more search parameters'
                )
            print('\n')

## Generate Data

In [9]:
# Generate resources
# result = runner.invoke(cli.generate,['./resources'])
# assert result.exit_code == 0

print('*********************')
print('Data Description')
print('*********************')
resources = load_resources(os.path.join(PROJECT_DIR, 'resources'))
df = pandas.DataFrame(
    [
        {'resource_type': r['resource_type'],
         'id': r['content'].get('id'),
         'references': r['content'].get('subject', {}).get('reference'),
        }
        for r in resources
    ]
)
display(df)
counts = df.groupby(['resource_type']).size()
display(counts)
# df[['resource_type', 'reference', 'id']]

2019-11-13 13:56:53,944 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00000-0.json
2019-11-13 13:56:53,947 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00001-0.json
2019-11-13 13:56:53,948 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00002-0.json
2019-11-13 13:56:53,950 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00003-0.json
2019-11-13 13:56:53,951 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00004-0.json
2019-11-13 13:56:53,952 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/projec

2019-11-13 13:56:54,066 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Specimen-BS-00009-0.json
2019-11-13 13:56:54,068 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Specimen-BS-00009-1.json


*********************
Data Description
*********************


,resource_type,id,references
0,Condition,CD-00000-0,Patient/PT-00000
1,Condition,CD-00001-0,Patient/PT-00001
2,Condition,CD-00002-0,Patient/PT-00002
3,Condition,CD-00003-0,Patient/PT-00003
4,Condition,CD-00004-0,Patient/PT-00004
5,Condition,CD-00005-0,Patient/PT-00005
6,Condition,CD-00006-0,Patient/PT-00006
7,Condition,CD-00007-0,Patient/PT-00007
8,Condition,CD-00008-0,Patient/PT-00008
9,Condition,CD-00009-0,Patient/PT-00009


resource_type
Condition      10
Observation    10
Patient        10
Specimen       20
dtype: int64

## Load Data

In [2]:
# Publish profiles and resources to server
for server_name in ['aidbox']:
    # Profiles
    server_settings = SERVER_CONFIG[server_name]
    print(f'\n\n********************** {server_name.upper()} **********************')
    username = server_settings.get('username')
    password = server_settings.get('password')    
    params = ['./profiles', '--resource_type', 'profile',
          '--server_name', server_name]
    if username and password:
        params.extend(['--username', username, '--password', password])    
    
    if server_name == 'aidbox':
        params.extend(['-e', 'search_parameter'])
        load_search_params(username, password)
        
    result = runner.invoke(cli.publish, params)
    assert result.exit_code == 0
    
    # Resources
    if server_name != 'vonk': # vonk has a bug in it right now - can't load twice        
        params = ['./resources', '--resource_type', 'resource',
              '--server_name', server_name]
        if username and password:
            params.extend(['--username', username, '--password', password])
    
        result = runner.invoke(cli.publish,params)
        assert result.exit_code == 0    
    print(f'\n\n*******************************************************************')

2019-11-13 13:35:48,259 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/search_parameters/us-core-race.json
2019-11-13 13:35:48,403 - kf_model_fhir.app - INFO - Begin publishing profile in ./profiles to Simplifier project http://localhost:8081/fhir




********************** AIDBOX **********************
Loading Aidbox SearchParameters ...
Success: PUT http://localhost:8081/SearchParameter/Patient.race


2019-11-13 13:35:49,527 - FhirValidator - INFO - Starting FHIR 4.0.0 profile validation for ./profiles
2019-11-13 13:35:49,529 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/extensions
2019-11-13 13:35:49,530 - kf_model_fhir.loader - WARNING - Skipping "extensions",  is an invalid resource file type
2019-11-13 13:35:49,531 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/Patient.json
2019-11-13 13:35:49,533 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/search_parameters
2019-11-13 13:35:49,533 - kf_model_fhir.loader - WARNING - Skipping "search_parameters",  is an invalid resource file type
2019-11-13 13:35:49,535 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/USCoreCondition.json
2019-11-13 

2019-11-13 13:35:49,683 - FhirApiClient - INFO - Begin deleting resources ...
2019-11-13 13:35:49,834 - FhirApiClient - DEBUG - GET http://localhost:8081/fhir/StructureDefinition?url=http://fhirr4.kids-first.io/fhir/StructureDefinition/us-core-condition succeeded. Response:
{'entry': [{'fullUrl': 'http://localhost:8081/StructureDefinition/us-core-condition',
            'resource': {'abstract': False,
                         'baseDefinition': 'http://hl7.org/fhir/StructureDefinition/Condition',
                         'contact': [{'telecom': [{'system': 'url',
                                                   'value': 'http://www.healthit.gov'}]}],
                         'date': '2019-05-21T00:00:00-04:00',
                         'derivation': 'constraint',
                         'description': 'Defines constraints and extensions on '
                                        'the Condition resource for the '
                                        'minimal set of data to query 

2019-11-13 13:35:49,837 - FhirApiClient - DEBUG - Fetched 1 item(s) from http://localhost:8081/fhir/StructureDefinition?url=http://fhirr4.kids-first.io/fhir/StructureDefinition/us-core-condition
2019-11-13 13:35:49,842 - FhirApiClient - DEBUG - Deleting http://localhost:8081/StructureDefinition/us-core-condition
2019-11-13 13:35:49,965 - FhirApiClient - DEBUG - DELETE http://localhost:8081/StructureDefinition/us-core-condition succeeded. Response:
{'abstract': False,
 'baseDefinition': 'http://hl7.org/fhir/StructureDefinition/Condition',
 'contact': [{'telecom': [{'system': 'url',
                           'value': 'http://www.healthit.gov'}]}],
 'date': '2019-05-21T00:00:00-04:00',
 'derivation': 'constraint',
 'description': 'Defines constraints and extensions on the Condition resource '
                'for the minimal set of data to query and retrieve problems '
                'and health concerns information.',
 'differential': {'element': [{'constraint': [{'expression': 'where(

...

2019-11-13 13:35:55,676 - FhirApiClient - INFO - ✅ PUT Observation-OB-00006-0.json to http://localhost:8081/fhir/Observation/OB-00006-0 succeeded
2019-11-13 13:35:55,677 - FhirApiClient - INFO - PUTing FHIR Observation from Observation-OB-00007-0.json
2019-11-13 13:35:55,714 - FhirApiClient - DEBUG - PUT http://localhost:8081/fhir/Observation/OB-00007-0 succeeded. Response:
{'code': {'coding': [{'code': '15074-8',
                      'display': 'Glucose [Moles/volume] in Blood',
                      'system': 'http://loinc.org'}]},
 'effectivePeriod': {'start': '2013-04-02T09:30:10+01:00'},
 'id': 'OB-00007-0',
 'identifier': [{'value': 'OB-00007-0'}],
 'interpretation': [{'coding': [{'code': 'H',
                                 'display': 'High',
                                 'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation'}]}],
 'issued': '2013-04-03T15:30:10+01:00',
 'meta': {'extension': [{'url': 'ex:createdAt',
                         'valueIn

2019-11-13 13:35:55,800 - FhirApiClient - INFO - ✅ PUT Observation-OB-00009-0.json to http://localhost:8081/fhir/Observation/OB-00009-0 succeeded
2019-11-13 13:35:55,800 - FhirApiClient - INFO - PUTing FHIR Condition from Condition-CD-00000-0.json
2019-11-13 13:35:55,840 - FhirApiClient - DEBUG - PUT http://localhost:8081/fhir/Condition/CD-00000-0 succeeded. Response:
{'abatementDateTime': '2016-01-28T03:52:16-05:00',
 'category': [{'coding': [{'code': 'encounter-diagnosis',
                           'display': 'Encounter Diagnosis',
                           'system': 'http://terminology.hl7.org/CodeSystem/condition-category'}]}],
 'clinicalStatus': {'coding': [{'code': 'resolved',
                                'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical'}]},
 'code': {'coding': [{'code': '271737000',
                      'display': 'Anemia',
                      'system': 'http://snomed.info/sct'}],
          'text': 'Anemia'},
 'id': 'CD-00000-0',
 'ide

2019-11-13 13:35:56,026 - FhirApiClient - INFO - ✅ PUT Condition-CD-00004-0.json to http://localhost:8081/fhir/Condition/CD-00004-0 succeeded
2019-11-13 13:35:56,028 - FhirApiClient - INFO - PUTing FHIR Condition from Condition-CD-00005-0.json
2019-11-13 13:35:56,069 - FhirApiClient - DEBUG - PUT http://localhost:8081/fhir/Condition/CD-00005-0 succeeded. Response:
{'abatementDateTime': '2016-01-28T03:52:16-05:00',
 'category': [{'coding': [{'code': 'encounter-diagnosis',
                           'display': 'Encounter Diagnosis',
                           'system': 'http://terminology.hl7.org/CodeSystem/condition-category'}]}],
 'clinicalStatus': {'coding': [{'code': 'resolved',
                                'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical'}]},
 'code': {'coding': [{'code': '87979003',
                      'display': 'Cleft Palate',
                      'system': 'http://snomed.info/sct'}],
          'text': 'Cleft Palate'},
 'id': 'CD-00005-0'

2019-11-13 13:35:56,247 - FhirApiClient - INFO - ✅ PUT Condition-CD-00009-0.json to http://localhost:8081/fhir/Condition/CD-00009-0 succeeded
2019-11-13 13:35:56,248 - kf_model_fhir.cli - INFO - ✅ Publish resource files succeeded!




*******************************************************************


# Search API Exploration

## Basic Queries

In [19]:
queries = [
    {
        'desc': 'Get all patients',
        'endpoint': 'Patient',
        'params': {}
    },
    {
        'desc': 'Get all female patients',            
        'endpoint': 'Patient',
        'params': {'gender': 'female'}
    },
    {
        'desc': 'Get all female patients with name containing Hol',        
        'endpoint': 'Patient',
        'params': {'gender': 'female', 'name:contains': 'Hol'}
    },
    {
        'desc': 'Get all female patients with last name = Holmes',        
        'endpoint': 'Patient',
        'params': {'gender': 'female', 'family': 'Holmes'}
    }
]
execute_queries(queries)

****************
Get all patients
****************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The query count = actual total Patient. This could be correct or the server returned all Patient because it did not understand one or more search parameters


VONK
----
Success: GET http://localhost:8080/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The query count = actual total Patient. This could be correct or the server returned all Patient because it did not understand one or more search parameters


HAPI
----
Success: GET https://try.smilecdr.com:8000/Patient
✅ Found 1007 Patient matching these params {'_total': 'accurate'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The query count = actual total Patient. This could be correct or the server returned all Patient 

## Using Modifiers

In [20]:
queries = [
    {
        'desc': 'Get all patients missing the gender attribute',
        'endpoint': 'Patient',
        'params': {'gender:missing': True}
    },
    {
        'desc': 'Get all patients that are NOT female',            
        'endpoint': 'Patient',
        'params': {'gender:not': 'female'}
    }
]
execute_queries(queries)

*********************************************
Get all patients missing the gender attribute
*********************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate', 'gender:missing': True}
⚠️ ？ The query count = actual total Patient. This could be correct or the server returned all Patient because it did not understand one or more search parameters


VONK
----
Success: GET http://localhost:8080/Patient
⚠️ or ✅ Found 0 Patient matching these params {'_total': 'accurate', 'gender:missing': True}


HAPI
----
Error with request. Status: 405. Caused by: {'issue': [{'code': 'processing',
            'diagnostics': ':missing modifier is disabled on this server',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET https://try.smilecdr.com:8000/Patient
❌ Found None Patient matching these params {'_total': 'accurate', 'gender:missing': True}


AZURE
-----
Success: G

## Comparison Operators

In [21]:
queries = [
    {
        'desc': 'Get all glucose in blood observations with value > 5 mmol/l',
        'endpoint': 'Observation',
        'params': {'value-quantity': 'gt5', 'code': '15074-8'}
    }
]
execute_queries(queries)

***********************************************************
Get all glucose in blood observations with value > 5 mmol/l
***********************************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Observation
✅ Found 4 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


VONK
----
Success: GET http://localhost:8080/Observation
✅ Found 10 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}
⚠️ ？ The query count = actual total Observation. This could be correct or the server returned all Observation because it did not understand one or more search parameters


HAPI
----
Success: GET https://try.smilecdr.com:8000/Observation
✅ Found 4 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Observation
✅ Found 2 Observation matching thes

## Searching Coded Things

In [22]:
queries = [
    {
        'desc': 'Get all specimens with code text = Left median cubital vein',
        'endpoint': 'Specimen',
        'params': {'bodysite:text': 'Left median cubital vein'}
    },
    {
        'desc': 'Get all specimens with bodysite code = 49852007 (Left median cubital vein)',            
        'endpoint': 'Specimen',
        'params': {'bodysite': '49852007'}
    },
    {
        'desc': 'Get all anemia conditions by code',            
        'endpoint': 'Condition',
        'params': {'code': '271737000'}
    }
]
execute_queries(queries)

***********************************************************
Get all specimens with code text = Left median cubital vein
***********************************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 11 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


VONK
----
Success: GET http://localhost:8080/Specimen
⚠️ or ✅ Found 0 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


HAPI
----
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 10 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Specimen
✅ Found 10 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


**************************************************************************
Get all specimens with bodysi

## Search Using References

In [23]:
# Get a sample patient name
from pprint import pprint
c = SERVER_CONFIG['vonk']
patient, sc = get(f"{c['base_url']}/Patient/PT-00001")
patient_name = patient['name'][0]['given'][0]

queries = [
    {
        'desc': 'Get all specimens for a patient by patient ID',
        'endpoint': 'Specimen',
        'params': {'subject:Patient': "PT-00001"}
    },
    {
        'desc': 'Get all specimens for a patient using their name (chained search parameters)',
        'endpoint': 'Specimen',
        'params': {'subject:Patient.name': patient_name}
    },
    {
        'desc': 'Get patients with anemia condition (reverse chained parameters)',
        'endpoint': 'Patient',
        'params': {'_has:Specimen:patient:bodysite': '49852007'}
    },
    {
        'desc': 'Get all the patients that have a Specimen where its Observation.status=final',
        'endpoint': 'Observation',
        'params': {'_has:Specimen:patient:_has:Observation:status': 'final'}
    },
]
execute_queries(queries)

Success: GET http://localhost:8080/Patient/PT-00001
*********************************************
Get all specimens for a patient by patient ID
*********************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


VONK
----
Success: GET http://localhost:8080/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


HAPI
----
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


AZURE
-----
Error with request. Status: 403. Caused by: {'id': 'ab29242d-b378-4d58-a56a-5c0d3f5a8300',
 'issue': [{'code': 'forbidden',
            'diagnostics': "Modifier 'Patient' is not supported for search "
                           "parameter 'subject'.",
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Succ

## Include Referenced Resources

In [24]:
queries = [
    {
        'desc': 'Get a specimens with its patient',
        'endpoint': 'Specimen',
        'params': {'identifier': "BS-00001-0",
         '_include': 'Specimen:patient'}
    },
    {
        'desc': 'Get an observation and the specimen it is about',
        'endpoint': 'Observation',
        'params': {'identifier': "OB-00001-0",
         '_include': 'Observation:specimen'}
    }
]
execute_queries(queries, display_content=True)


********************************
Get a specimens with its patient
********************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
{'entry': [{'fullUrl': 'http://localhost:8081/Patient/PT-00001',
            'resource': {'extension': [{'extension': [{'url': 'text',
                                                       'valueString': 'American '
                                                                      'Indian '
                                                                      'or '
                                                                      'Alaska '
                                                                      'Native'},
                                                      {'url': 'ombCategory',
                                                       'valueCoding': {'code': '1002-5',
                                                                       'display': 'American '
                                            

Success: GET https://try.smilecdr.com:8000/Specimen
{'entry': [{'fullUrl': 'https://try.smilecdr.com:8000/Specimen/BS-00001-0',
            'resource': {'collection': {'bodySite': {'coding': [{'code': '49852007',
                                                                 'display': 'Structure '
                                                                            'of '
                                                                            'median '
                                                                            'cubital '
                                                                            'vein '
                                                                            '(body '
                                                                            'structure)',
                                                                 'system': 'http://snomed.info/sct'}],
                                                     'text': 'Left median '
     

                         'id': 'BS-00001-1',
                         'identifier': [{'value': 'BS-00001-1'}],
                         'meta': {'lastUpdated': '2019-11-13T17:45:30.596+00:00',
                                  'profile': ['http://hl7.org/fhir/StructureDefinition/Specimen'],
                                  'versionId': '626dccdd-7072-45fd-ba95-27898b02b580'},
                         'resourceType': 'Specimen',
                         'subject': {'reference': 'http://localhost:8080/Patient/PT-00001'}},
            'search': {'mode': 'include'}},
           {'fullUrl': 'http://localhost:8080/OperationOutcome/226a946f-c0a7-4ad5-ad07-4e100fbf05f3',
            'resource': {'id': '226a946f-c0a7-4ad5-ad07-4e100fbf05f3',
                         'issue': [{'code': 'not-supported',
                                    'details': {'coding': [{'code': '5003',
                                                            'system': 'http://hl7.org/fhir/dotnet-api-operation-outcome

## Full Text Search

In [25]:
queries = [
    {
        'desc': 'Full text search - get all specimens w/ code text = Left median cubital vein',
        'endpoint': 'Specimen',
        'params': {'_content': '"Left median cubital vein"'}
    }
]
execute_queries(queries)

****************************************************************************
Full text search - get all specimens w/ code text = Left median cubital vein
****************************************************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 11 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


VONK
----
Success: GET http://localhost:8080/Specimen
✅ Found 20 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}
⚠️ ？ The query count = actual total Specimen. This could be correct or the server returned all Specimen because it did not understand one or more search parameters


HAPI
----
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 26 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Specimen
✅ Found 20 Spec

## Custom Search Parameter

In [26]:
queries = [
    {
        'desc': 'Search on extension - get all patients with particular race',
        'endpoint': 'Patient',
        'params': {'race': '2028-9'}
    }
]
execute_queries(queries)

***********************************************************
Search on extension - get all patients with particular race
***********************************************************
AIDBOX
------
Error with request. Status: 500. Caused by: {'id': 'exception',
 'issue': [{'code': 'exception',
            'diagnostics': 'Search for {:resourceType :Patient, :type :param, '
                           ':name "race", :values [{:value "2028-9"}], '
                           ':search-param #object[clojure.lang.Agent 0x5c8fc75 '
                           '{:status :ready, :val nil}]} is not implemented',
            'severity': 'fatal'}],
 'resourceType': 'OperationOutcome',
 'text': {'div': 'Search for {:resourceType :Patient, :type :param, :name '
                 '"race", :values [{:value "2028-9"}], :search-param '
                 '#object[clojure.lang.Agent 0x5c8fc75 {:status :ready, :val '
                 'nil}]} is not implemented',
          'status': 'generated'}}
Success: GET http:/

## Custom Search Query

## Exact Path Match

In [27]:
queries = [
    {
        'desc': 'Get all specimens with analyte type = DNA',
        'endpoint': 'Specimen',
        'params': {'.extension.0.valueString': 'DNA'}
    }
]
execute_queries(queries)

*****************************************
Get all specimens with analyte type = DNA
*****************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 8 Specimen matching these params {'.extension.0.valueString': 'DNA', '_total': 'accurate'}


VONK
----
Success: GET http://localhost:8080/Specimen
✅ Found 20 Specimen matching these params {'.extension.0.valueString': 'DNA', '_total': 'accurate'}
⚠️ ？ The query count = actual total Specimen. This could be correct or the server returned all Specimen because it did not understand one or more search parameters


HAPI
----
Error with request. Status: 400. Caused by: {'issue': [{'code': 'processing',
            'diagnostics': 'Unknown search parameter "". Value search '
                           'parameters for this search are: [_id, _language, '
                           'accession, bodysite, collected, collector, '
                           'container, container-id, identifier, parent, '
   